<a href="https://colab.research.google.com/github/rpn1966/hello-world/blob/main/LaneEmdenWidget.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install mpld3
import numpy as np
import matplotlib.pyplot as plt
import mpld3
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from IPython.display import display, Markdown, Latex, Math
mpld3.enable_notebook()

In [9]:
def plotter(x,y):
  import matplotlib.pyplot as plt
  plt.plot(x,y,'b')
  plt.title(r'$\alpha$ versus $\beta$')

In [10]:
@widgets.interact_manual(dxi=[0.001, 0.01, 0.1],n=[0,1.,1.5,2.,3.,3.3,4.,5.],Mstar=[0.1,0.5,1.0,2.0,5.0],Rstar=[0.1,0.5,1.0,2.0,5.0])
def lane_emden(dxi,n,Mstar,Rstar):
  from IPython.display import display, Markdown, Latex, Math
  # Define physical constants required to define the stellar models
  Pi=np.pi
  G=6.67E-11
  k_B=1.38E-23
  m_H=1.67E-27
  mu=0.62
  
  # Solar values
  Msun=1.989E30
  Rsun=7.E8
  Lsun2=3.86E26
  
  # Define maximum number of integration points
  Nmax=100000
  
  # Ask the user to input the polytropic index, n
  #n=float(input('Enter n, the polytrope index '))
  
  # Ask the user to input physical parameters of the star
  #Rstar=float(input('Enter Rstar, the radius of the star in units of the Solar radius '))
  #Mstar=float(input('Enter Mstar, the mass of the star in units of the Solar mass '))
  Rstar=Rstar*Rsun     # Convert radius to physical units
  Mstar=Mstar*Msun     # Convert mass to physical units
  
  # The variables are as follows 
  # Dimensionless length : xi 
  # Dimensionless density function : theta
  # The gradient of theta wrt xi : dtheta_dxi
  
  # Here we define floating point arrays for xi, theta, dtheta_dxi:
  xi=np.zeros(Nmax,dtype=np.float64)
  theta=np.zeros(Nmax,dtype=np.float64)
  dtheta_dxi=np.zeros(Nmax,dtype=np.float64)
  
  # Specify boundary conditions, and start the integration a small distance (1.E-6) 
  # away from the centre of the star (remember that starting at exactly xi=0.0 is not 
  # possible since the equations diverge there through division by zero.
  theta[0]=1.0
  dtheta_dxi[0]=0.0
  xi[0]=1.E-6
  
  # The step size in radius is defined by dxi.
  # The smaller the step the more accurate the calculation.
  # And the longer it takes!
  #dxi = 0.001
  
  # We now define a variable, i, that counts the number of steps as we move out 
  # through the star. We cycle around a loop, computing the values at each 
  # radius point in the star until the value of theta becomes negative, 
  # (or until we iterate through the maximum number of grid points allowed by Nmax), 
  # at which point the integration ends and we jump out of the loop.
  
  i=0    # Note that i=0 corresponds to the radius point at the centre of the star

  while theta[i] > 0 and i < Nmax-1:    
    i=i+1   # Increment i during each iteration of the loop
    
    # Calculate the next value of dtheta_dxi at radius point i using data from i-1
    dtheta_dxi[i]=dtheta_dxi[i-1] - ( 2.0*dtheta_dxi[i-1]/xi[i-1] + theta[i-1]**n )*dxi
    
    # Calculate the next value of theta[i] using the updated value of dtheta_dxi[i]
    theta[i] = theta[i-1] + dtheta_dxi[i]*dxi 
    
    # Increment the dimensionless radius (move out to the next radius point in the star).
    xi[i] = xi[i-1] + dxi
    
    # Save the final positive value of theta and its gradient for determining the
    # value of xi where theta=0
    thetalast = theta[i-1]
    dtheta_dxilast = dtheta_dxi[i-1]
  
  # Note we are now out of the loop as we have changed the indentation!
  
  # We calculate the position of the root (xi=xi_1), using the last
  # value of theta to be computed (which  is the first value < 0, and the  
  # previous theta value (stored as thetalast) which is the last value > 0.
  # To find xi_1 we interpolate between the last positive value and first 
  # negative value of theta to estimate the location where theta=0.
  # We do the same for dtheta_dxi to find the value of the gradient
  # at the location xi=xi_1.
  xi_1 = xi[i] + (theta[i]/(thetalast-theta[i]))*dxi
  dtheta_dxi_1 = dtheta_dxi[i] - (xi[i]-xi_1)*(dtheta_dxi[i]-dtheta_dxilast)/dxi
  print('')
  print ('The value of xi_1= ',xi_1) 
  print ('At this point dtheta/dxi = ',dtheta_dxi_1)
  display(Math(r'\phi'))
  plotter(xi[0:i-1],theta[0:i-1])
  #plt.plot(xi[0:i-1],theta[0:i-1],'b-')
  #plt.title(r'$\theta$ versus $\xi$')
  #print('')
  #display(Latex(r'\mathrm{\mathbf{The \, value \, of} \, \xi_1 \, \mathbf{is:}}'), xi_1)
  #print('')
  #display(Latex(r'\mathrm{\mathbf{The \, value \, of} \, \left(\frac{d\theta}{d \xi}\right)_{\xi_1} \, \mathbf{is:}}'),dtheta_dxi_1)



interactive(children=(Dropdown(description='dxi', options=(0.001, 0.01, 0.1), value=0.001), Dropdown(descripti…